In [ ]:
import pandas as pd

# Carregar os dados das planilhas
df_atendimentos = pd.read_excel('exportação\\atendimentos_04-12-2024.xlsx')  # Substitua pelo nome do seu arquivo
df_protocolos_excel = pd.read_excel('exportação\\dados_excel_04-12-2024.xlsx')  # Substitua pelo nome do seu arquivo

# Função para normalizar os nomes das colunas
def normalize_column_names(df):
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_', regex=False)
    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    return df

# Aplicar a função aos DataFrames
df_atendimentos = normalize_column_names(df_atendimentos)
df_protocolos_excel = normalize_column_names(df_protocolos_excel)

# Exibir os novos nomes das colunas
print("Colunas de df_atendimentos:", df_atendimentos.columns)
print("Colunas de df_protocolos_excel:", df_protocolos_excel.columns)

# Converter colunas de datas para o tipo datetime
df_atendimentos['data'] = pd.to_datetime(df_atendimentos['data'], errors='coerce')
df_protocolos_excel['data'] = pd.to_datetime(df_protocolos_excel['data'], errors='coerce')

# Remover espaços em branco nos campos de texto e converter para string
df_atendimentos['codigo_da_guia'] = df_atendimentos['codigo_da_guia'].astype(str).str.strip()
df_protocolos_excel['numero_da_guia'] = df_protocolos_excel['numero_da_guia'].astype(str).str.strip()

df_atendimentos['beneficiario'] = df_atendimentos['beneficiario'].astype(str).str.strip().str.upper()
df_protocolos_excel['beneficiario'] = df_protocolos_excel['beneficiario'].astype(str).str.strip().str.upper()

# Normalizar o campo 'carteira' (se ainda for relevante)
df_atendimentos['carteira'] = df_atendimentos['carteira'].astype(str).str.strip()
df_protocolos_excel['carteira'] = df_protocolos_excel['carteira'].astype(str).str.strip()

# Converter 'assinatura' para booleano, Se os valores forem 'Sim' e 'Não'
df_atendimentos['assinatura'] = df_atendimentos['assinatura'].map({'Sim': True, 'Não': False})

# Agrupar os protocolos por 'numero_da_guia', 'data' e 'beneficiario' para contar as execuções
protocolo_counts = df_protocolos_excel.groupby(['numero_da_guia', 'data', 'beneficiario']).size().reset_index(name='quantidade_assinaturas_esperadas')

# Iniciar a lista de divergências
divergencias = []

In [ ]:
# Iterar sobre cada grupo no protocolo_counts
for index, row in protocolo_counts.iterrows():
    idGuia = row['numero_da_guia']
    dataExec = row['data']
    beneficiario_paciente = row['beneficiario']
    quantidade_esperada = row['quantidade_assinaturas_esperadas']

    # Filtrar atendimentos correspondentes
    atendimentos_filtrados = df_atendimentos[
        (df_atendimentos['codigo_da_guia'] == idGuia) &
        (df_atendimentos['data'] == dataExec) &
        (df_atendimentos['beneficiario'] == beneficiario_paciente)
    ]

    quantidade_real = len(atendimentos_filtrados)

    problemas = []

    # Verificar se as quantidades correspondem
    if quantidade_real != quantidade_esperada:
        problemas.append('Quantidade de assinaturas divergente')

    # Verificar datas não preenchidas nos atendimentos
    datas_nao_preenchidas = atendimentos_filtrados['data'].isnull().sum()
    if datas_nao_preenchidas > 0:
        problemas.append('Datas não preenchidas nos atendimentos')

    # Verificar número da guia não preenchido nos atendimentos
    guias_nao_preenchidas = atendimentos_filtrados['codigo_da_guia'].isnull().sum()
    if guias_nao_preenchidas > 0:
        problemas.append('Número da guia não preenchido nos atendimentos')

    # Verificar assinaturas ausentes nos atendimentos
    # Certifique-se de que a coluna 'assinatura' seja do tipo booleano ou adequada para a comparação
    assinaturas_ausentes = atendimentos_filtrados['assinatura'] == False
    if assinaturas_ausentes.any():
        problemas.append('Assinaturas ausentes nos atendimentos')

    # Verificar se há datas preenchidas acima da quantidade executada
    if quantidade_real > quantidade_esperada:
        problemas.append('Datas preenchidas acima da quantidade executada')

    # Se houver problemas, adicionar às divergências
    if problemas:
        divergencia = {
            'numero_da_guia': idGuia,
            'data': dataExec.strftime('%Y-%m-%d') if pd.notnull(dataExec) else None,
            'beneficiario': beneficiario_paciente,
            'quantidade_assinaturas_esperadas': quantidade_esperada,
            'quantidade_assinaturas_encontradas': quantidade_real,
            'problemas': ', '.join(problemas)
        }
        divergencias.append(divergencia)

# Converter a lista de divergências em um DataFrame
divergencias_df = pd.DataFrame(divergencias)

# Exibir as divergências encontradas
print(divergencias_df)

# Salvar as divergências em um arquivo Excel
divergencias_df.to_excel('divergencias.xlsx', index=False)


### Criar Retangulos ao Redor dos Campos Desejados

In [173]:
# Detecção automática de objetos

import cv2

# Carregar a imagem
image_path = "page_1.png"
image = cv2.imread(image_path)

# Converter para escala de cinza
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Aplicar um limiar (threshold) para binarizar a imagem
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# Encontrar contornos na imagem binarizada
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar retângulos ao redor dos campos desejados
for contour in contours:
    # Obter as coordenadas do retângulo delimitador
    x, y, w, h = cv2.boundingRect(contour)
    
    # Filtrar os campos com base no tamanho (ex: evitar ruídos ou elementos irrelevantes)
    if 50 < w < 800 and 20 < h < 100:  # Ajuste esses valores com base na sua imagem
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Retângulo verde

# Salvar ou exibir a imagem processada
output_path = "output_image.png"
cv2.imwrite(output_path, image)

print("Imagem processada salva em:", output_path)


Imagem processada salva em: output_image.png


In [174]:
# Desenho manual de retângulos ao redor dos campos desejados

# Carregar a imagem
image_path = "page_1.png"
image = cv2.imread(image_path)


# Lista de coordenadas dos retângulos e seus rótulos com cores
# Formato: [(x, y, largura, altura, "Nome do campo", (B, G, R)), ...]
rectangles = [
    (100, 330, 210, 706, "DATA ATENDIMENTO", (255, 0, 0)),  # Azul
    (335, 340, 330, 688, "CARTEIRA", (255, 0, 0)),  # Azul
    (690, 340, 360, 688, "BENEFICIARIO", (255, 0, 0)),  # Azul
    (1075, 340, 220, 688, "NUMERO GUIA", (255, 0, 0)),  # Azul
    (1400, 340, 240, 688, "ASSINATURA", (255, 0, 0)),  # Azul
    # Adicione mais coordenadas específicas com seus respectivos nomes e cores
]

# Desenhar os retângulos e adicionar os textos
for rect in rectangles:
    x, y, w, h, label, color = rect
    # Desenhar o retângulo com a cor especificada
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    
    # Adicionar o texto acima do retângulo
    text_x = x + 30
    text_y = y - 7  # Colocar o texto um pouco acima do retângulo
    cv2.putText(image, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Salvar ou exibir a imagem processada
output_path = "output_image.png"
cv2.imwrite(output_path, image)

print("Imagem processada salva em:", output_path)

Imagem processada salva em: output_image.png


In [171]:
# Desenho manual de retângulos ao redor dos campos desejados com fundo no texto personalizado
import cv2

# Carregar a imagem
image_path = "page_1.png"
image = cv2.imread(image_path)

print(image_path)

# Lista de coordenadas dos retângulos e seus rótulos com cores
# Formato: [(x, y, largura, altura, "Nome do campo"), ...]
rectangles = [
    (1400, 75, 210, 45, "    1. FICHA  "),
    (100, 332, 210, 697, "    2. DATA  "),
    (345, 340, 320, 688, "     3. CARTEIRA        "),
    (690, 340, 360, 688, " 4. BENEFICIARIO           "),
    (1075, 340, 220, 688, " 5. NUMERO GUIA"),
    (1400, 340, 240, 688, "  6. ASSINATURA "),
    # Adicione mais coordenadas específicas com seus respectivos nomes
]

# Retangulo com fundo opaco 
cv2.rectangle(image, (1, 1), (1400, 323), (255, 255, 255), -1)
cv2.rectangle(image, (1400, 150), (1700, 340), (255, 255, 255), -1)



# Cor das caixas e textos (vermelho)
box_color = (0, 0, 255)  # Vermelho em formato (B, G, R)
text_color = (0, 0, 255)  # Vermelho para a fonte
text_bg_color = (255, 255, 255)  # Branco para o fundo do texto

# Desenhar os retângulos e adicionar os textos com fundo personalizado
for rect in rectangles:
    x, y, w, h, label = rect

    # Desenhar o retângulo ao redor do campo
    cv2.rectangle(image, (x, y), (x + w, y + h), box_color, 5)

    # Dimensões do texto
    text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
    text_width, text_height = text_size[0], text_size[1]

    # Coordenadas do retângulo de fundo do texto (mais espaço na horizontal e vertical)
    padding = 3 # Controle de espaço ao redor do texto
    text_bg_x1, text_bg_y1 = x + 0, y - text_height - 20 - padding
    text_bg_x2, text_bg_y2 = x + 30 + text_width + 20, y - 5 + padding

    # Desenhar retângulo de fundo branco para o texto
    cv2.rectangle(image, (text_bg_x1, text_bg_y1), (text_bg_x2, text_bg_y2), text_bg_color, -1)  # Fundo branco

    # Adicionar o texto em vermelho sobre o fundo branco
    cv2.putText(image, label, (text_bg_x1 + 0, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)  # Texto vermelho

# Salvar ou exibir a imagem processada
output_path = "output_image.png"
cv2.imwrite(output_path, image)

print("Imagem processada salva em:", output_path)


page1.png


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [165]:

import pymupdf
def pdf_to_images(pdf_path, output_folder, dpi=300):
    # Abrir o PDF
    doc = pymupdf.open(pdf_path)

    # Converter cada página em uma imagem
    for page_number in range(len(doc)):
        page = doc[page_number]
        pix = page.get_pixmap(dpi=dpi)
        
        # Salvar a imagem
        output_path = f"{output_folder}/page_{page_number + 1}.png"
        pix.save(output_path)
        print(f"Página {page_number + 1} salva em {output_path}")

    doc.close()

# Exemplo de uso
pdf_path = "guias_novas\\Document_20240312_0001.pdf"
output_folder = "guias_processadas"
pdf_to_images(pdf_path, output_folder)

ModuleNotFoundError: No module named 'pymupdf'